# Finding trials registered on ClinicalTrials.gov that do not have reported results

Reporting of clinical trial results became mandatory for many trials in 2008. However [this paper](http://www.bmj.com/content/352/bmj.i637) and [this investigation](https://www.statnews.com/2015/12/13/clinical-trials-investigation/) both find that substantial numbers of clinical trials have not reported results, even for those trials where the FDAAA has made reporting mandatory.

This notebook examines how many trials on ClinicalTrials.gov have had their results publicly reported. We have a broader definition of a trial that should report its results than the FDAAA. We count a trial as eligible for our analysis if:

- it has overall status of 'Completed'
- it has a study type of 'Interventional' 
- its completion date was after 1 Jan 2006, but is more than 24 months ago
- it is phase 2 or later (or its phase is N/A, ie it's a trial of a device or a behavioural intervention)
- it has no results disposition date (i.e. no application to delay results has been filed).

We then classify it as overdue if it has no summary results attached on ClinicalTrials.gov, and no results on PubMed that are linked by NCT ID (see below). 

This is substantially broader than FDAAA, which covers only US-based trials of FDA-approved drugs. However, we think all trials should report their results, not just US-based trials, or FDA-approved drugs. In addition, FDAAA requires results to be reported within 12 months of completion, and we allow 24 months.

ClinicalTrials.gov supplies notes on [how to find studies with results](https://clinicaltrials.gov/ct2/help/how-find/find-study-results) and [results in general](https://clinicaltrials.gov/ct2/about-site/results).

In [1]:
import csv
from datetime import datetime
from dateutil.relativedelta import relativedelta
import glob
from pprint import pprint
import requests
import StringIO
import time
import zipfile

import numpy as np
import pandas as pd
from pyquery import PyQuery as pq
from slugify import slugify
from xml.etree import ElementTree

## Obtain the raw XML data

First, we get the raw XML trial summaries from ClinicalTrials.gov - this is supplied as a [single very large zip file](https://clinicaltrials.gov/search?studyxml=true), containing more than 200,000  XML files. ClinicalTrials.gov supplies [field definitions](https://prsinfo.clinicaltrials.gov/definitions.html).

In [2]:
# Set this to True if you want to download the latest raw data. 
# TODO: make sure this code works, rather than doing it manually!
REDOWNLOAD_XML = False
if REDOWNLOAD_XML:
    r = requests.get('https://clinicaltrials.gov/search?studyxml=true', stream=True)
    if not r.ok:
        print 'Problem downloading'
    zip_ref = zipfile.ZipFile(StringIO.StringIO(r.content))
    zip_ref.extractall('./data/search_result')
    zip_ref.close()
print 'done'

done


## Create summary results file

Extract the fields of interest from the XML summaries, and save them to a CSV file, which we'll use as our source data for the rest of this exercise. Note that this section is skipped by default, for the purposes of development. 

In [3]:
fname = 'trials.csv'
REGENERATE_SUMMARY = False
if REGENERATE_SUMMARY:
    files = glob.glob('./data/search_result/*.xml')
    fieldnames = ['nct_id', 'title', 'overall_status', 
                  'study_type', 'completion_date',
                  'lead_sponsor', 'lead_sponsor_class',
                  'collaborator', 'collaborator_class', 
                  'phase', 'locations', 'has_drug_intervention', 'drugs', 
                  'disposition_date', 'results_date', 
                  'enrollment']
    trials = csv.DictWriter(open(fname, 'wb'), fieldnames=fieldnames)
    trials.writeheader()
    for i, f in enumerate(files):
        if i % 50000 == 0:
            print i, f
#         print i, f
        text = open(f, 'r').read()
        d = pq(text, parser='xml')
        data = {}
        for f in fieldnames:
            data[f] = None
        
        data['nct_id'] = d('nct_id').text()
        data['title'] = d('brief_title').text().strip()
        data['overall_status'] = d('overall_status').text().strip()
        data['phase'] = d('phase').text().replace("Phase ", "")
        
        data['lead_sponsor'] = d('lead_sponsor agency').text()
        data['lead_sponsor_class'] = d('lead_sponsor agency_class').text()
        data['collaborator'] = d('collaborator')('agency').text()
        data['collaborator_class'] = d('collaborator')('agency_class').text()
        
        data['study_type'] = d('study_type').text()
        data['completion_date'] = d('primary_completion_date').text()
        data['results_date'] = d('firstreceived_results_date').text()
        # Not used, see below for discussion of this field.
        # data['results_pmids'] = d('results_reference PMID').text()  
        data['enrollment'] = d('enrollment').text()
        
        # Not currently used, but might be useful in future. 
        data['has_drug_intervention'] = False
        data['drugs'] = ''
        for it in d('intervention'):
            e = pq(it)
            if e('intervention_type').text() == 'Drug':
                data['has_drug_intervention'] = True
                data['drugs'] += e('intervention_name').text() + '; '
                
        data['disposition_date'] = d('firstreceived_results_disposition_date').text()
        data['locations'] =  d('location_countries country').text()

        for k in data:
            if data[k] and isinstance(data[k], basestring):
                data[k] = data[k].encode('utf8')
        trials.writerow(data)
        
print 'done'

done


## Load data for analysis

Normalise date fields, and load into Pandas.  

In [4]:
dtype = {'has_drug_intervention': bool, 
         'phase': str } 
datefields = ['completion_date', 'results_date', 'disposition_date']
df = pd.read_csv(fname,
                 parse_dates=datefields, 
                 infer_datetime_format=True,
                 dtype=dtype)
print len(df), 'trials found'

227798 trials found


In [5]:
df.tail()

,nct_id,title,overall_status,study_type,completion_date,lead_sponsor,lead_sponsor_class,collaborator,collaborator_class,phase,locations,has_drug_intervention,drugs,disposition_date,results_date,results_pmids,enrollment
227793,NCT02934282,HBOC-201 Expanded Access Protocol for Life-thr...,Temporarily not available,Expanded Access,NaT,University of Miami,Other,NaN,NaN,NaN,NaN,True,HBOC-201;,NaT,NaT,NaN,NaN
227794,NCT02934295,Study of Rubella Immunity. Response to Vaccina...,"Active, not recruiting",Interventional,2014-05-01,Hopital Foch,Other,NaN,NaN,NaN,France,False,NaN,NaT,NaT,NaN,192.0
227795,NCT02934308,Comparison of the Skin Conductance Values and ...,Recruiting,Interventional,2017-09-01,Hopital Foch,Other,NaN,NaN,NaN,France,False,NaN,NaT,NaT,NaN,60.0
227796,NCT02934321,Assessment of Satiety Following Oral Administr...,Not yet recruiting,Interventional,2017-08-01,University of Florida,Other,NaN,NaN,NaN,United States,False,NaN,NaT,NaT,NaN,25.0
227797,NCT02934334,Wellness Monitoring for Major Depressive Disorder,Enrolling by invitation,Observational,2018-12-01,Sidney Kennedy,Other,NaN,NaN,NaN,Canada,False,NaN,NaT,NaT,NaN,100.0


In [6]:
def normalise_phase(x):
    # Set N/A (trials without phases, e.g. device trials) to 5 (i.e. later than 
    # phase 2, which is our cutoff for inclusion). And set phase 1/2 trials to 1.
    if pd.isnull(x):
        x = 5
    return int(str(x).split('/')[0])
assert normalise_phase(None) == 5
assert normalise_phase('3') == 3
assert normalise_phase('1/2') == 1
df['phase_normalised'] = df['phase'].apply(normalise_phase)
df.phase_normalised.value_counts()

5    102483
2     40150
1     33776
3     26721
4     22759
0      1909
Name: phase_normalised, dtype: int64

### Calculate whether trials are completed

The criteria for counting a trial as completed are defined above.  

In [7]:
startdate = datetime.strptime('01 January 2006', '%d %B %Y')
cutoff = datetime.now() - relativedelta(years=2)
# cutoff = datetime.strptime('28 September 2014', '%d %B %Y') 
print 'Cutoff date', cutoff

df['is_completed'] = (df.overall_status == 'Completed') & \
    (df.completion_date >= startdate) & \
    (df.completion_date <= cutoff) & \
    (df.phase_normalised >= 2) & \
    (df.disposition_date.isnull() & \
    (df.study_type.str.startswith('Interventional')))
df['is_overdue'] = (df.is_completed & \
                    df.results_date.isnull())
                    # & df.results_pmids.isnull())
df_completed = df[df.is_completed] 
df_overdue = df[df.is_completed & df.results_date.isnull()]
print len(df), 'total trials found'
print len(df_completed), 'are completed and due results, by our definition'
print len(df[df.is_completed & ~df.results_date.isnull()]), \
    'trials due results have submitted results on CT.gov'
print len(df_overdue), \
    'trials due results have not submitted results on CT.gov'
# print int(df_completed.enrollment.sum()), 'total patients enrolled in completed trials'
# print int(df_overdue.enrollment.sum()), 'total patients enrolled in overdue trials'

Cutoff date 2014-10-21 13:35:59.519152
227798 total trials found
48966 are completed and due results, by our definition
14304 trials due results have submitted results on CT.gov
34662 trials due results have not submitted results on CT.gov


## Check for results on PubMed

If trials have reported their results on PubMed, and if it's possible [to find them on PubMed using a linked NCT ID](https://www.nlm.nih.gov/bsd/policy/clin_trials.html), then we count those trials as having submitted results. 

So, for all trials that we regard as completed and due results, search PubMed using these two IDs, and look for that has been anything published between the completion date and now, that don't have the words "study protocol" in the title, and that are [classified as results of a trial (using the clinical keyword "therapy")](https://www.ncbi.nlm.nih.gov/books/NBK3827/#pubmedhelp.Clinical_Queries_Filters). 

At the time of writing, about 14,000 of the 48,000 completed trials have results on PubMed. An example of a trial with results on PubMed: [NCT02460380](https://www.ncbi.nlm.nih.gov/pubmed/?term=NCT00000619%5BSecondary%20Source%20ID%5D%20AND%20(%222010%2F07%2F01%22%5BPDAT%5D%20%3A%20%223000%22%5BPDAT%5D)&cmd=DetailsSearch} (TODO: Update this). 

Note 1: we know from the BMJ studies that there are trials that do have results on PubMed, but that aren't linked using the NCT ID. The BMJ authors found these using a manual search. Some examples: `NCT00002762: 19487378`, `NCT00002879: 18470909`, `NCT00003134: 19066728`, `NCT00003596: 18430910`. We regard these as invalid, because you can only find results via an exhaustive manual search. We only count results as published for our purposes if they are either (i) submitted on ClinicalTrials.gov or (ii) retrievable on PubMed using the NCT ID. 

Note 2: we know there are some trials that have results PMIDs directly in ClinicalTrials.gov, in the `results_reference` field of the XML. After discussion with Jess here, and Annice at ClinicalTrials.gov, I decided that these results are too often meaningless to be useful - lots of the time they aren't truly results, but are studies from years ago.

In [8]:
def get_pubmed_title(pmid):
    '''
    Retrieve the title of a PubMed article, from its PMID.
    '''
    url = 'http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?'
    url += 'db=pubmed&rettype=abstract&id=%s' % pmid
    try:
        resp = requests.get(url)
    except ValueError, requests.ConnectionError:
        print 'Error!', url
        time.sleep(10)
        return get_pubmed_title(pmid)
    try:
        tree = ElementTree.fromstring(resp.content)
        title = tree.find('.//Article/ArticleTitle')
        if title is not None:
            title = title.text.encode('utf8')
    except ElementTree.ParseError:
        print 'ParseError', url
        title = ''
    return title

def get_pubmed_linked_articles(nct_id, completion_date, query_type):
    '''
    Given an NCT ID, search PubMed for related results articles.
    '''
    url = 'http://eutils.ncbi.nlm.nih.gov/entrez/eutils/'
    url += 'esearch.fcgi?db=pubmed&retmode=json&term='
    url += '(%s[si] OR %s[Title/Abstract]) ' % (nct_id, nct_id)
    url += 'AND ("%s"[pdat] : ' % completion_date.strftime('%Y/%m/%d')
    url += '"3000"[pdat])'
#     print url
    if query_type == 'broad':
        url += "AND ((clinical[Title/Abstract] AND trial[Title/Abstract]) OR clinical trials as topic[MeSH Terms] OR clinical trial[Publication Type] OR random*[Title/Abstract] OR random allocation[MeSH Terms] OR therapeutic use[MeSH Subheading])"
    elif query_type == 'narrow':
        url += "AND (randomized controlled trial[Publication Type] OR (randomized[Title/Abstract] AND controlled[Title/Abstract] AND trial[Title/Abstract]))"     
    try:
        resp = requests.get(url)
        data = resp.json()
    except (ValueError, requests.ConnectionError) as e:
        print 'Error!', url
        time.sleep(10)
        return get_pubmed_linked_articles(nct_id, completion_date, query_type)
    esearchresult = data['esearchresult']
    ids = []
    if 'idlist' in esearchresult:
        ids = esearchresult['idlist']
        for id1 in ids[:]:
            title = get_pubmed_title(id1)
            if title and 'study protocol' in title.lower():
                ids.remove(id1)
    return ids

import sqlite3
conn = sqlite3.connect('trials-title.db')
cur = conn.cursor()  
c = "CREATE TABLE IF NOT EXISTS trials(nct_id TEXT PRIMARY KEY, "
c += "pubmed_results INT, pubmed_results_broad INT, pubmed_results_narrow INT)"
cur.execute(c)
conn.commit()

REGENERATE_PUBMED_LINKS = False

# Only bother looking for PubMed results for overdue trials
# (but set the value on the original dataframe).
df['pubmed_results'] = False
count = 0
for i, row in df_overdue.iterrows():
    if count % 1000 == 0:
        print count, row.nct_id
    count += 1
    
#     pubmed_results = False
    # First, check for results stored locally.
    c = "SELECT nct_id, pubmed_results, pubmed_results_broad, pubmed_results_narrow FROM trials WHERE nct_id='%s'" % row.nct_id
    cur.execute(c)
    data = cur.fetchone()
#     data = False
    if data: 
        simple_results = int(data[1])
#         if REGENERATE_PUBMED_LINKS and data[1]:
#                 pubmed_results = bool(data[1])
#         else:
#             pubmed_results = bool(data[1])
    else:
        # No local results, check PubMed.
#         print row.nct_id, row.completion_date
        simple_results = get_pubmed_linked_articles(row.nct_id, row.completion_date, '')
        broad_results = get_pubmed_linked_articles(row.nct_id, row.completion_date, 'broad')
        narrow_results = get_pubmed_linked_articles(row.nct_id, row.completion_date, 'narrow')
#     c = "INSERT OR REPLACE INTO trials VALUES('%s', %s, %s, %s)" % \
#         (row.nct_id, len(simple_results), len(broad_results), len(narrow_results))
#     cur.execute(c)
#     conn.commit()
    has_simple_results = simple_results > 0
    df.set_value(i, 'pubmed_results', has_simple_results) 
    
cur.close()
conn.close()
print df[df.is_completed & df.results_date.isnull()].pubmed_results.value_counts()
print 'done' 

0 NCT00000176
1000 NCT00094432
2000 NCT00156026
3000 NCT00220402
4000 NCT00280475
5000 NCT00345917
6000 NCT00407095
7000 NCT00471965
8000 NCT00535756
9000 NCT00604357
10000 NCT00661310
11000 NCT00719342
12000 NCT00779493
13000 NCT00834951
14000 NCT00900718
15000 NCT00956215
16000 NCT01009112
17000 NCT01065246
18000 NCT01121692
19000 NCT01179074
20000 NCT01237795
21000 NCT01297725
22000 NCT01357265
23000 NCT01421043
24000 NCT01482273
25000 NCT01544868
26000 NCT01609491
27000 NCT01681719
28000 NCT01754077
29000 NCT01826409
30000 NCT01912430
31000 NCT02010320
32000 NCT02128464
33000 NCT02301598
34000 NCT02599181
False    25739
True      8923
Name: pubmed_results, dtype: int64
done


### Calculate final overdue count

Now we have looked for PubMed results, we can calculate the final overdue count.

In [9]:
df['is_overdue'] = (df.is_completed & df.results_date.isnull() & ~df.pubmed_results)
# Reset df_completed, because df has new fields that we need to include.
df_completed = df[df.is_completed]
print len(df_completed), 'trials should have published results'
print df.is_overdue.value_counts()
df_overdue = df[df.is_overdue]
print len(df_overdue), 'trials have not published results'
percent_submitted = (1 - (len(df_overdue) / float(len(df_completed)))) * 100
print '%s%% of completed trials have published results' % \
    '{:,.2f}'.format(percent_submitted)
print int(df_overdue.enrollment.sum()), 'total patients are enrolled in overdue trials'

48966 trials should have published results
False    202059
True      25739
Name: is_overdue, dtype: int64
25739 trials have not published results
47.43% of completed trials have published results
9211069 total patients are enrolled in overdue trials


## Write to CSV

Output final results to a CSV file, which we will use in the interactive version. We reshape the data so it has a row for each sponsor, and columns by year - two columns for each year, one for the number of completed trials with overdue results, and one for the total completed trials. 

In [10]:
# We're only interested in the completed trials, and we will divide the data up by year.
df_completed['year_completed'] = df_completed['completion_date'].dt.year.dropna().astype(int)
df_completed['year_completed'] = df_completed.year_completed.astype(int)

# Drop all sponsors with fewer than N completed trials.
NUM_TRIALS = 5
df_final = df_completed[
    df_completed.groupby('lead_sponsor').nct_id.transform(len) > NUM_TRIALS]

# Now reshape the data: a row for each sponsor, columns by year:
# lead_sponsor,2008_overdue,2008_total,2009_overdue,2009_total...
df_temp = df_final.set_index(['lead_sponsor', 'year_completed']) 
gb = df_temp.groupby(level=[0, 1]).is_overdue
df2 = gb.agg({'overdue': 'sum', 'total': 'count'}) \
          .unstack().swaplevel(0, 1, 1).sort_index(1)
df2.columns = df2.columns.to_series().apply(lambda x: '{}_{}'.format(*x))
df3 = df2.reset_index()
df3['lead_sponsor_slug'] = df3.lead_sponsor.apply(slugify)
df3.to_csv('../app/data/completed.csv', index=None)
print len(df3), 'sponsors found at %s trials' % NUM_TRIALS

# Write the raw output to a full spreadsheet. 
df.to_csv('all.csv', index=None)

1341 sponsors found at 5 trials


/Library/Python/2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Library/Python/2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


## For refernece: Compare our results with BMJ authors

A [2016 BMJ paper](http://www.bmj.com/content/352/bmj.i637) found that around 65% of papers reprted results. "Overall, 2892 of the 4347 clinical trials (66.5%) had been published or reported results as of July 2014." 

Excellently, the BMJ authors [publish their raw data on DataDryad](http://datadryad.org/resource/doi:10.5061/dryad.6n018) so we can compare our results with theirs, to get an idea of the difference between our automated strategy and their partially manual strategy. (However, in their reported data it looks to me like the matched PMID rate is 59.9% of all NCT IDs.)

The BMJ authors were looking at a much smaller set of papers than us, because they focussed on academic medical centres. Their set is slightly different, because they include pre-Phase-2 trials. They also used a manual search strategy which involved searching Scopus and manually comparing results. 

In [63]:
from openpyxl import load_workbook
import sys
bmj_results = load_workbook(filename = './chen-bmj.xlsx')

In [90]:
nct_ids = {}
count = 0
has_pmid = 0

# The Excel data has multiple worksheets, sigh. 
# And NCT IDs can occur more than once with different results, sigh.
# We only care about where there's at least one result.
for sheet in bmj_results.worksheets:
    for i, row in enumerate(sheet.rows):
        if i == 0:
            continue
        if row[0].value:
            count += 1
#             print row[0].value, row[6].value, type(row[6].value)
            if isinstance(row[6].value, long):
                val = str(row[6].value)
            else:
                val = row[6].value
#             print row[0].value, val
            if val:
                has_pmid += 1
            # Always set val if it exists.
            # Otherwise, only set val if there's no existing value.
            if val:
                nct_ids[row[0].value] = val
            else:
                if not row[0].value in nct_ids:
                    nct_ids[row[0].value] = val



print count, 'rows found in total'
print has_pmid, 'of those rows have a PMID'
print has_pmid / float(count) * 100, 'per cent of their NCT IDs have a PMID, including duplicates'
print
unique_nct_ids = len(nct_ids.keys())
print unique_nct_ids, 'unique NCT IDs found in all rows'
pmids_found = sum(1 for x in nct_ids.values() if x)
print pmids_found, 'of these have PMIDs'
print pmids_found / float(unique_nct_ids) * 100, 'per cent of their NCT IDs have a PMID, not including duplicates'

4347 rows found in total
2452 of those rows have a PMID
56.4067172763 per cent of their NCT IDs have a PMID, including duplicates

4092 unique NCT IDs found in all rows
2295 of these have PMIDs
56.0850439883 per cent of their NCT IDs have a PMID, not including duplicates


### Compare with our data

Now examine: 

* of the NCT IDs for which BMJ authors find PubMed results, how many we also find PubMed results for
* of the same dataset, how many *only BMJ* find results for
* of the NCT IDs for which BMJ authors do not find PubMed results, how many we do find PubMed results

In [91]:
df_bmj = pd.Series(nct_ids).to_frame(name='pmid')
df_bmj['pubmed_results'] = ~df_bmj.pmid.isnull()
df_bmj.index.name = 'nct_id'
df_bmj.reset_index(inplace=True)
print len(df_bmj), 'NCT IDs in the full BMJ dataset'
# df_bmj.head(20)

merged_results = \
    pd.merge(df_bmj, df_completed, #[['nct_id', 'pubmed_results']], 
             on='nct_id', how='inner', suffixes=('_bmj', '_ours'))
    
# NB I tried this first with a left join: but 1521 out of the 4500 papers 
# just don't appear in our dataset. This seems to be because the BMJ
# paper includes earlier-phase trials. To get a sample after a left join...
# merged_results[merged_results.pubmed_results_ours.isnull()].head()

merged_results.pubmed_results_ours.value_counts(dropna=False)
# merged_results.head()
print len(merged_results), 'NCT IDs are in both the BMJ dataset and ours'
papers_both_find_pm_results = \
    merged_results[merged_results.pubmed_results_bmj & merged_results.pubmed_results_ours]
papers_both_find_pm_results.head()
print len(papers_both_find_pm_results), 'we both find results for'
papers_only_they_find_results = \
    merged_results[merged_results.pubmed_results_bmj & ~merged_results.pubmed_results_ours]
print len(papers_only_they_find_results), 'only they find results for'  
papers_only_we_find_results = \
    merged_results[~merged_results.pubmed_results_bmj & merged_results.pubmed_results_ours]
print len(papers_only_we_find_results), 'only we find results for'

4092 NCT IDs in the full BMJ dataset
2561 NCT IDs are in both the BMJ dataset and ours
369 we both find results for
1277 only they find results for
42 only we find results for


In [94]:
cols = ['nct_id', 'title', 'pubmed_results_bmj', 'pmid', 'pubmed_results_ours']
papers_only_they_find_results.head()[cols]

,nct_id,title,pubmed_results_bmj,pmid,pubmed_results_ours
1,NCT00000620,Action to Control Cardiovascular Risk in Diabe...,True,18539917,False
2,NCT00002684,Combination Chemotherapy in Patients With Adva...,True,17320659,False
3,NCT00002717,Paclitaxel and Cisplatin in Treating Patients ...,True,17906207,False
4,NCT00002879,Cladribine in Patients With Mantle Cell Lymphoma,True,18470909,False
5,NCT00003134,Irinotecan in Treating Patients With Recurrent...,True,19066728,False
